In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.metrics as mtr

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Classificazione rank
Il rank è un valore tra 0 e 20 calcolato nello script ranks.ipynb.
In questo script ho due metodi per gestire i nomi delle squadre: 
- utilizzo dummy 
- utilizzo i codici 

In [10]:
dataset = pd.read_csv('../SerieA/Season21_22/matches.csv', index_col=0)
rank = pd.read_csv('../SerieA/Season21_22/rank.csv', index_col=0)
#dataset = dataset[['h_team', 'a_team','result']]
dataset = dataset.drop(columns=['date', 'matchday', 'h_goals', 'a_goals', 'id'])

In [11]:
for i, ds in dataset.iterrows():
    row = rank.loc[rank['Team'] == ds.h_team]
    val = row['Rank'].values[0]
    dataset.loc[i, 'Rank_H'] = val

    row = rank.loc[rank['Team'] == ds.a_team]
    val = row['Rank'].values[0]
    dataset.loc[i, 'Rank_A'] = val

## Dummy

In [4]:
stacked = dataset[['h_team', 'a_team']].stack()
index = stacked.index.get_level_values(0)
result = pd.crosstab(index=index, columns=stacked)
result.index.name = None
result.columns.name = None

dummy_match = pd.get_dummies(dataset
        ,columns = ['h_team']
        ,prefix = ['h']
        )

dummy_match = dummy_match.drop(['a_team'], axis=1)
pos = 0
for col in result.columns:
    dummy_match.insert(pos, col, result[col])
    pos += 1

dummy_match['rank_h'] = dataset.Rank_H
dummy_match['rank_a'] = dataset.Rank_A
dummy_match['result'] = dataset.result
dataset = dummy_match

## Codici associati

In [12]:
dataset["home_code"] = dataset["h_team"].astype("category").cat.codes
dataset["away_code"] = dataset["a_team"].astype("category").cat.codes
dataset = dataset.drop(columns=['h_team', 'a_team'])

## Universale (continuo sia di Dummy che Codici associati)

In [13]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=10000), #max_iter di default vale 100, ho dovuto alzarlo se no non converge
    'Support Vector Machine': SVC(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

In [14]:
features = [x for x in dataset.columns if x != 'result']
X, y = dataset[features], dataset.result.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
for model_name, model in models.items():
    models[model_name].fit(X_train, y_train)

predictions = {}
for model_name, model in models.items():
    predictions[model_name] = model.predict(X_test)

In [16]:
E = []
for estimator, y_pred in predictions.items():
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    E.append({
        'Model': estimator, 'Accuracy': report['accuracy'],
        'Avg Precision (macro)': report['macro avg']['precision'],
        'Avg Recall (macro)': report['macro avg']['recall'],
        'Avg F1-score (macro)': report['macro avg']['f1-score'],
        'Avg Precision (weighted)': report['weighted avg']['precision'],
        'Avg Recall (weighted)': report['weighted avg']['recall'],
        'Avg F1-score (weighted)': report['weighted avg']['f1-score']
    })
E = pd.DataFrame(E).set_index('Model', inplace=False)

In [17]:
E

,Accuracy,Avg Precision (macro),Avg Recall (macro),Avg F1-score (macro),Avg Precision (weighted),Avg Recall (weighted),Avg F1-score (weighted)
Model,,,,,,,
Logistic Regression,0.644737,0.598990,0.580527,0.585266,0.652432,0.644737,0.643436
Support Vector Machine,0.407895,0.335784,0.371493,0.263757,0.407217,0.407895,0.300365
Multinomial Naive Bayes,0.500000,0.473314,0.455948,0.457502,0.489069,0.500000,0.489119
Decision Tree,0.473684,0.472222,0.470188,0.461802,0.510417,0.473684,0.483077
Random Forest,0.618421,0.562570,0.562570,0.562570,0.618421,0.618421,0.618421
K-Nearest Neighbors,0.407895,0.384419,0.385241,0.383731,0.415658,0.407895,0.410574
